In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import re
from time import sleep

input_file = "23oct.csv"
output_file = "result_final.csv"

data = pd.read_csv(input_file)
driver = webdriver.Firefox()

pattern_lines = r"(?:^|\s)(first|second|third|fourth|fifth|sixth|seventh|eighth|ninth|tenth)(?:\samendment)?"
pattern_amendment_no = r"\s*?amendment\s*no\.?\s*(\d+)"

def extracting_amendment_number(url):
    try:
        driver.get(url)
        sleep(5)
        page_text = driver.find_element(By.TAG_NAME, 'body').text.lower()
        lines_10 = page_text.splitlines()[:10]
        
        for line in lines_10:
            match_lines = re.search(pattern_lines, line)
            match_amendment_no = re.search(pattern_amendment_no, line)

            if match_lines:
                amendment_name = match_lines.group(1)
                if amendment_name:
                    amendment_number = {
                        "first": "1",
                        "second": "2",
                        "third": "3",
                        "fourth": "4",
                        "fifth": "5",
                        "sixth": "6",
                        "seventh": "7",
                        "eighth": "8",
                        "ninth": "9",
                        "tenth": "10"
                    }.get(amendment_name)
                    if amendment_number:
                        return amendment_number
            elif match_amendment_no:
                amendment_number = match_amendment_no.group(1)
                return amendment_number

        return "None"
    except Exception as e:
        return f"Error reading URL: {str(e)}"

result_data = pd.DataFrame(columns=["SEC.gov URL", "Amendment Number"])

for index, row in data.iterrows():
    amendment_number = extracting_amendment_number(row["SEC.gov URL"])
    result_data = pd.concat([result_data, pd.DataFrame({"SEC.gov URL": [row["SEC.gov URL"]], "Amendment Number": [amendment_number]})], ignore_index=True)

driver.quit()

result_data.to_csv(output_file, index=False)
print("DONE! Results saved to result_final.csv.")

DONE! Results saved to result_final.csv.
